In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
from IPython.display import Image, display

import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)

In [2]:
from typing import Annotated, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages

model = llm

class State(TypedDict):
    foo: str

class SubgraphState(TypedDict):
    # tidak ada kunci ini yang dibagikan dengan status grafik induk
    bar: str
    baz: str

# Definisikan subgrafik
def subgraph_node(state: SubgraphState):
    return {"bar": state["bar"] + "baz"}

subgraph_builder = StateGraph(SubgraphState)
subgraph_builder.add_node(subgraph_node)
 
subgraph = subgraph_builder.compile()

# Definisikan grafik induk
def node(state: State):
    # ubah status ke status subgrafik
    response = subgraph.invoke({"bar": state["foo"]})
    # ubah respons kembali ke status induk
    return {"foo": response["bar"]}

builder = StateGraph(State)
# perhatikan bahwa kita menggunakan fungsi `node` alih-alih subgrafik yang dikompilasi
builder.add_node(node)
 
graph = builder.compile()
png_bytes =graph.get_graph().draw_mermaid_png()
display(Image(png_bytes))

ValueError: Graph must have an entrypoint: add at least one edge from START to another node